# MPIN Strength Checker

I have implemented each logic in separate parts to get clarity.
- Commonly used PINs
- Demographic-based PINs 
- Strength evaluation with reason



## Part A: Load Commonly Used PINs

This section loads the top N commonly used PINs from a .csv file. These PINs are sourced from [SecLists](https://github.com/danielmiessler/SecLists), a curated collection of security-related lists. The dataset was selected after thorough research to ensure it represents real-world patterns.

In [10]:
import pandas as pd

def load_top_pins(pin_file_path, top_n=100):
    try:
        df = pd.read_csv(pin_file_path, header=None, names=["PIN", "Frequency"])
        df["PIN"] = df["PIN"].astype(str).str.strip()
        topPins = df.sort_values(by="Frequency", ascending=False).head(top_n)["PIN"].tolist()
        return topPins
    except FileNotFoundError:
        print("Error: PIN file not found.")
        return []
    except Exception as e:
        print(f"Error: {e}")
        return []

## Part B: Generate Demographic PINs

This section generates possible PIN combinations based on user demographics, such as:
- Date of Birth (DOB).
- Spouse's Date of Birth.
- Wedding Anniversary.

The generated PINs include patterns like DDMMYY, MMDDYY, YYDDMM etc.

In [11]:
def generate_demographic_pins(dob, spouse_dob, anniversary):
    pins = {
        "dob_self":set(),
        "dob_spouse":set(),
        "anniversary":set()
    }
    def permutations(date):
        if len(date) == 8:  #DDMMYYYY
            dd = date[:2]
            mm = date[2:4]
            yyyy = date[4:]
            yy = date[6:]

            return {dd + mm + yy, mm + dd + yy, yyyy[:2] + dd + mm, yyyy[:2] + mm + dd}
        return set()
    if dob:
        pins["dob_self"].update(permutations(dob))
    if spouse_dob:
        pins["dob_spouse"].update(permutations(spouse_dob))
    if anniversary:
        pins["anniversary"].update(permutations(anniversary))
    
    return pins

## Part C: Evaluate PIN Strength

i then found out the strength by referring to
- Commonly used PINs.
- Demographic-based PINs.

The program returns:
- *Strength: WEAK or STRONG.
- *Reasons for Weakness:
  - COMMONLY_USED
  - DEMOGRAPHIC_DOB_SELF
  - DEMOGRAPHIC_DOB_SPOUSE
  - DEMOGRAPHIC_ANNIVERSARY

In [12]:
def pinStrength(pin,common_pins,demographic_pins):
    reasons = []
    if pin in common_pins:
        reasons.append("COMMONLY_USED")
    if pin in demographic_pins["dob_self"]:
        reasons.append("DEMOGRAPHIC_DOB_SELF")
    if pin in demographic_pins["dob_spouse"]:
        reasons.append("DEMOGRAPHIC_DOB_SPOUSE")
    if pin in demographic_pins["anniversary"]:
        reasons.append("DEMOGRAPHIC_ANNIVERSARY")
    
    # Determine strength
    if reasons:
        return "WEAK", reasons
    else:
        return "STRONG", []

## Part D: Main Program

This section combines all parts to evaluate the strength of a user-provided 6-digit PIN.

In [13]:
# Main Program
if __name__ == "__main__":
    
    pin_file_path = "/Users/dev/Desktop/OneBanc_assignment_devyani/most_common_pins/6_digit_pins.csv"
    commonPins = load_top_pins(pin_file_path)
    print("loaded common PINs:", commonPins)

    
    dob = input("Enter your DOB(DDMMYYYY): ")
    spouse_dob = input("Enter your Spouse's DOB(DDMMYYYY): ")
    anniversary = input("Enter your Wedding Anniversary(DDMMYYYY): ")
    demographic_pins = generate_demographic_pins(dob, spouse_dob, anniversary)
    print("generated demographic PINs:", demographic_pins)

    pin = input("Enter PIN to evaluate its strength: ")
    strength, reasons = pinStrength(pin, commonPins, demographic_pins)
    print(f"Strength: {strength}")
    print(f"Reasons: {reasons}")

loaded common PINs: ['PIN', '444444', '321123', '889900', '040404', '000000', '111111', '147258', '222222', '666666', '112233', '090909', '456654', '789987', '987654', '050505', '445566', '070707', '333333', '121212', '999999', '777777', '123456', '789123', '789456', '101010', '778899', '456789', '123321', '334455', '010101', '123789', '654321', '080808', '888888', '223344', '555555', '060606', '654456', '556677', '667788', '987789', '020202', '258369', '030303']
generated demographic PINs: {'dob_self': {'050104', '200105', '010504', '200501'}, 'dob_spouse': {'201903', '200319', '190310', '031910'}, 'anniversary': {'190610', '061910', '201906', '200619'}}
Strength: WEAK
Reasons: ['DEMOGRAPHIC_DOB_SELF']
